<a href="https://colab.research.google.com/github/yeeeeeun/GEC_by_edit_spans/blob/main/final_gec_propose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from transformers import Trainer, TrainingArguments
import json

## GPU 부족으로 Kaggle 사용


In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
# https://huggingface.co/j5ng/et5-base

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BartConfig,PreTrainedTokenizerFast
import torch
# Load the model and tokenizer
model_name = 'gogamza/kobart-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda:0")

In [ ]:
model = model.to(device)

In [ ]:
with open('/kaggle/input/ffffffinal-dataset/propose_processed_data.json', "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

df_sampled = df.sample(n=30000, random_state=42)

x = df_sampled['input']
y = df_sampled['output']

train_df, test_df = train_test_split(df_sampled, train_size=20000, test_size=10000, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

x_train = train_df['input']
x_valid = val_df['input']
y_train = train_df['output']
y_valid = val_df['output']

print("Train 데이터 크기:", train_df.shape[0])
print("Valid 데이터 크기:", val_df.shape[0])
print("Test 데이터 크기:", test_df.shape[0])

Train 데이터 크기: 16000
Valid 데이터 크기: 4000
Test 데이터 크기: 10000


In [ ]:
print(train_df.head())

                        input                                     output
118839    꼭 아침에 밥 먹고 알바가리~!!!  (13, 13, ' '), (14, 15, ''), (17, 19, '')
79677   아몬드 페페는 이 삼일에 주라고 하드라  (9, 10, ''), (19, 20, '더'), (21, 21, '.')
215391                등등 많이주는                                (5, 5, ' ')
78682                 야채죽...?                                 (3, 6, '')
176584                     네네                                (2, 2, '.')


In [ ]:

max_length = df_sampled.iloc[:, 0].str.len().max()


longest_sentence = df_sampled.loc[df_sampled.iloc[:, 0].str.len() == max_length, df_sampled.columns[0]].values[0]

print("가장 긴 문장의 글자수:", max_length)
print("가장 긴 문장을 출력함:", longest_sentence)

가장 긴 문장의 글자수: 477
가장 긴 문장을 출력함: 그러면서도 자기가 딸에게 다시 잘이야기 해보신다며 여운을 남기시길래 집오면서 도와달라고 말씀을 드렸고, 다음날엔 저렇게 카톡을 보내시길래 제가 이런 문자를 보냈거든
네. 어머니 말씀 잘이해했습니다. 어제 제게 일말의 여운을 남기지 않으셨다면 하루더빨리 마음정리 할 수 있었을껍니다.  어머니께서 좋은마음 갖고 도와주시겠다고 하시어 드린말씀이지 집착은 아니었습니다. 매주 따님과 저희 집에서 함께지내고, 또 매일 붙어 있다싶이 했습니다. 때론 어머니께 거짓말을하고 여행을 가기도 했습니다. 이젠 서로 마음이 깊어졌고 그 깊어진 마음 보여드리려했을뿐 현실적인것을 말씀드리려한건 아닙니다. 돈이 중요한거 알고 안정적인 생활이 중요한건 압니다. 제가 부모님께 도움을 받았으면 받았지 드리는 형편아닙니다. 부모님은 언론인 출신이시고 공무원생활도 오래하신 분들 입니다. 현실보다 감정이 기반된 사랑을 하고싶었을 뿐이에요. 아쉽지만 저는 이만 하렵니다. 잘지내세요.


In [ ]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer

class GECDatasets(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.input = [example['input'] for example in data]
        self.output = [example['output'] for example in data]  # 수정 정보
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.input[idx]
        edits = self.output[idx]


        encoding = self.tokenizer(input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        output_encoding = self.tokenizer(edits, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        labels = output_encoding['input_ids'].squeeze(0)

        sample = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

        return sample


In [ ]:
dataset = GECDatasets(data=df_sampled.to_dict(orient='records'), tokenizer=tokenizer)

In [ ]:
print(dataset[0])

{'input_ids': tensor([14427,  9866, 20141, 12258, 14304,   214,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
            3,     3,     3,     3,     3,     3, 

In [ ]:
print(len(dataset))

30000


In [ ]:
train_data = []
for input_text, edits in zip(x_train.values, y_train.values):
    train_data.append({
        'input': input_text,  # 잘못된 문장
        'output': edits        # 수정 정보
    })

valid_data = []
for input_text, edits in zip(x_valid.values, y_valid.values):
    valid_data.append({
        'input': input_text,  # 잘못된 문장
        'output': edits        # 수정 정보
    })

train_dataset = GECDatasets(train_data, tokenizer)
valid_dataset = GECDatasets(valid_data, tokenizer)

In [ ]:
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback 설정
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.0,
)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=50,
    report_to="none",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    callbacks=[early_stopping],
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.078800,0.076171
2,0.067400,0.065905
3,0.053900,0.060678
4,0.043300,0.058898
5,0.032700,0.060018
6,0.026800,0.065547


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=12000, training_loss=0.06755638842781385, metrics={'train_runtime': 2188.2691, 'train_samples_per_second': 73.117, 'train_steps_per_second': 9.14, 'total_flos': 7316837498880000.0, 'train_loss': 0.06755638842781385, 'epoch': 6.0})

In [ ]:
wrong_input = '그는 사과을 먹고있다.'

input_ids = tokenizer(wrong_input, return_tensors='pt').input_ids
edit_out = model.generate(input_ids, max_length=128, num_beams=4, length_penalty=2.0, early_stopping=True)
corrected_output = tokenizer.decode(edit_out[0], skip_special_tokens=True)

edits = get_edit_operations(wrong_input, corrected_output)
print(edits)

In [ ]:
torch.save(model.state_dict(), './final_gec_model')

In [ ]:
import numpy as np
from IPython.display import FileLink
import os

result = np.array([0.0])
np.save("/kaggle/working/embedding",result)
os.chdir(r'/kaggle/working')
FileLink(r'./final_gec_model')

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = 'gogamza/kobart-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.load_state_dict(torch.load('/kaggle/working/final_gec_model', map_location=torch.device('cpu')))
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print("문장을 입력하세요:")
input_text = input()

input_encoding = tokenizer(input_text, return_tensors="pt").to(device)

output_encoding = model.generate(
    input_ids=input_encoding.input_ids,
    attention_mask=input_encoding.attention_mask,
    max_length=128,
    num_beams=5,
)

output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

print("edit spans: ", output_text)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/tmp/ipykernel_30/3546478252.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary

문장을 입력하세요:


 왔어용?


edit spans:  (2, 3, '요')


In [ ]:
!pip install m2-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement m2-score (from versions: none)
ERROR: No matching distribution found for m2-score


In [ ]:
import torch
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import time
import ast

model_name = 'gogamza/kobart-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.load_state_dict(torch.load('/kaggle/working/final_gec_model', map_location=torch.device('cpu')))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

test_df = test_df.sample(n=1000, random_state=42)

def apply_edit_spans(input_sentence, edit_spans):

    input_sentence = list(input_sentence)
    try:
        for span in sorted(edit_spans, key=lambda x: x[0], reverse=True):
            start, end, replacement = span
            input_sentence[start:end] = list(replacement)
    except Exception as e:
        print(f"Error in apply_edit_spans: {e}")
        print(f"edit_spans: {edit_spans}")
        raise e
    return ''.join(input_sentence)

def calculate_bleu(model, tokenizer, dataset, max_length=128):
    references = []
    hypotheses = []

    start_time = time.time()
    with torch.no_grad():
        for _, row in tqdm(dataset.iterrows(), desc="Calculating BLEU", total=len(dataset)):
            source_sentence = row['input']
            target_sentence = row['output']

            inputs = tokenizer(source_sentence, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)

            output = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=max_length)
            raw_edit_spans = tokenizer.decode(output[0], skip_special_tokens=True)

            try:
                edit_spans = ast.literal_eval(f"[{raw_edit_spans}]")
            except Exception as e:
                print(f"Error parsing edit spans: {e}")
                print(f"raw_edit_spans: {raw_edit_spans}")
                continue

            corrected_sentence = apply_edit_spans(source_sentence, edit_spans)

            references.append([target_sentence.split()])
            hypotheses.append(corrected_sentence.split())

    prediction_time = time.time() - start_time
    bleu_score = corpus_bleu(references, hypotheses)
    return bleu_score, prediction_time

bleu_score, prediction_time = calculate_bleu(model, tokenizer, test_df)
print(f"BLEU Score: {bleu_score}")
print(f"Time: {prediction_time}")


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/tmp/ipykernel_30/2550538412.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary

Error parsing edit spans: invalid syntax (<unknown>, line 1)
raw_edit_spans: (3,, ' '), (15, 15, ' '), (17, 17, ' '), (22, 22, '. ')


Calculating BLEU:   8%|▊         | 76/1000 [00:50<10:11,  1.51it/s]

Error parsing edit spans: invalid syntax (<unknown>, line 1)
raw_edit_spans: (3,, ' '), (4, 4, ' '), (6, 6, ' '), (11, 11, ' '), (13, 13, ' '), (16, 16, ' '), (17, 17, '.')


Calculating BLEU:  13%|█▎        | 126/1000 [01:22<09:39,  1.51it/s]

...

Calculating BLEU:  98%|█████████▊| 977/1000 [10:42<00:15,  1.52it/s]

Error parsing edit spans: invalid syntax (<unknown>, line 1)
raw_edit_spans: (2,, '찮아.'), (11, 12, ''), (17, 17, '.')


Calculating BLEU: 100%|██████████| 1000/1000 [10:57<00:00,  1.52it/s]

BLEU Score: 0.05479911274499472
Time: 657.9792716503143



/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
